In [2]:
import gradio as gr
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
import pickle

# Load cell lines data and top genes
cell_lines = pd.read_csv('data/gene_expression.csv', index_col=0)
with open('data/2128_genes.pkl', 'rb') as f:
    top_genes = pickle.load(f)

# Load model
with open('trained_models/xgboost.pkl', 'rb') as f:
    model = pickle.load(f)

filtered_cell_lines = cell_lines[cell_lines.columns.intersection(top_genes)]

# Define the smiles_to_fingerprint function
def smiles_to_fingerprint(smiles):
    mol = Chem.MolFromSmiles(smiles)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024)
    return fp

# Define a function that will be called when the user makes a prediction
def predict(smiles_notation):
    # Transform SMILES to fingerprint
    fingerprint = smiles_to_fingerprint(smiles_notation)
    # print(fingerprint)

    # Convert the fingerprint to a DataFrame with one row and columns representing bits
    fingerprint_df = pd.DataFrame([list(fingerprint)], columns=range(1024)).apply(lambda x: pd.Series({f'fp{str(i)}': val for i, val in enumerate(x)}), axis=1)

    # Merge the fingerprint with each row of filtered_cell_lines
    fingerprint_df['common_key'] = 1
    filtered_cell_lines['common_key'] = 1
    merged_data = pd.merge(filtered_cell_lines, fingerprint_df, on='common_key').drop('common_key', axis=1)

    # Perform any additional processing or prediction based on the merged_data
    predicts = model.predict(merged_data)

    #create dataset with two columns: cell line and IC50


    #merge predicts with cell lines (label columns = 'Cell_line', 'IC50')
    predicts = pd.DataFrame({'IC50': predicts,
                             'Cell_line': filtered_cell_lines.index})

    #sort by IC50 (only lowest 20)
    predicts = predicts.sort_values(by='IC50').head(10)

    return predicts

# Define the Gradio interface
iface = gr.Interface(
    fn=predict,  # Placeholder for now, as the function will be defined dynamically
    inputs=gr.Textbox(value="COc1cc(O)c2c(c1)C=CCC(O)C(O)C(=O)C=CCC(C)OC2=O", lines=1, label="Enter drug in SMILES notation"),
    outputs=gr.Dataframe(headers=['IC50', 'Cell_line'],type="numpy",label = 'Top 10 Cell Lines with lowest IC50' , datatype="number", row_count=10, col_count=2),  # Adjust as needed
    title="Drug Response Prediction",
)


# Launch the Gradio interface
iface.launch(share=True)


Running on local URL:  http://127.0.0.1:7861

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


C:\Users\Paulo Seixal\AppData\Local\Temp\ipykernel_31508\4030054351.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_cell_lines['common_key'] = 1


In [83]:
with open('trained_models/xgboost.pkl', 'rb') as file:
    model = pickle.load(file)

In [124]:


def predict(smiles_notation):
    # Transform SMILES to fingerprint
    fingerprint = smiles_to_fingerprint(smiles_notation)
    # print(fingerprint)

    # Convert the fingerprint to a DataFrame with one row and columns representing bits
    fingerprint_df = pd.DataFrame([list(fingerprint)], columns=range(1024)).apply(lambda x: pd.Series({f'fp{str(i)}': val for i, val in enumerate(x)}), axis=1)

    # Merge the fingerprint with each row of filtered_cell_lines
    fingerprint_df['common_key'] = 1
    filtered_cell_lines['common_key'] = 1
    merged_data = pd.merge(filtered_cell_lines, fingerprint_df, on='common_key').drop('common_key', axis=1)

    # Perform any additional processing or prediction based on the merged_data
    predicts = model.predict(merged_data)

    #create dataset with two columns: cell line and IC50


    #merge predicts with cell lines (label columns = 'Cell_line', 'IC50')
    predicts = pd.DataFrame({'IC50': predicts,
                             'Cell_line': filtered_cell_lines.index})

    #sort by IC50 (only lowest 20)
    predicts = predicts.sort_values(by='IC50').head(10)

    return predicts

In [125]:
predict('COc1cc(O)c2c(c1)C=CCC(O)C(O)C(=O)C=CCC(C)OC2=O')

C:\Users\Paulo Seixal\AppData\Local\Temp\ipykernel_20640\1239822744.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_cell_lines['common_key'] = 1


,IC50,Cell_line
302,0.180740,EoL-1-cell
763,0.826552,MOLM-13
403,1.116237,MV-4-11
448,1.708198,RS4-11
551,1.739541,CML-T1
542,1.743071,A4-Fuk
352,1.744109,KE-37
399,1.781451,MONO-MAC-6
447,1.868176,RPMI-8402
775,1.868289,SU-DHL-5
